In [2]:
import numpy as np
import logging
from typing import Tuple, List
from datetime import datetime
import matplotlib.pyplot as plt


# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    datefmt="%m-%d %H:%M:%S",
)


def validate_inputs(A: np.ndarray, b: np.ndarray, x0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
   
    # Convertir A a numpy array
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)
    
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A debe ser cuadrada (n×n)")
    
    # Convertir b a numpy array
    if not isinstance(b, np.ndarray):
        b = np.array(b, dtype=float)
    
    if b.ndim == 1:
        b = b.reshape(-1, 1)
    
    if b.shape[0] != A.shape[0]:
        raise ValueError("El vector b debe tener n elementos")
    
    # Convertir x0 a numpy array
    if not isinstance(x0, np.ndarray):
        x0 = np.array(x0, dtype=float)
    
    if x0.ndim == 1:
        x0 = x0.reshape(-1, 1)
    
    if x0.shape[0] != A.shape[0]:
        raise ValueError("El vector x0 debe tener n elementos")
    
    return A, b, x0


def gauss_jacobi(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                tol: float = 1e-6, max_iter: int = 100) -> Tuple[np.ndarray, List[np.ndarray]]:
    
    A, b, x0 = validate_inputs(A, b, x0)
    
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    
    logging.info(f"Iteración 0: x = {x.flatten()}")
    
    for k in range(1, max_iter + 1):
        x_new = np.zeros((n, 1))
        
        for i in range(n):
            # Suma de todos los elementos excepto el diagonal
            suma = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - suma) / A[i, i]
        
        # Verificar convergencia
        if np.linalg.norm(x_new - x) < tol:
            logging.info(f"Convergencia alcanzada en {k} iteraciones")
            return x_new, trajectory
        
        x = x_new.copy()
        trajectory.append(x.copy())
        logging.info(f"Iteración {k}: x = {x.flatten()}")
    
    logging.warning(f"No se alcanzó convergencia después de {max_iter} iteraciones")
    return x, trajectory


def gauss_seidel(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                tol: float = 1e-6, max_iter: int = 100) -> Tuple[np.ndarray, List[np.ndarray]]:

    A, b, x0 = validate_inputs(A, b, x0)
    
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    
    logging.info(f"Iteración 0: x = {x.flatten()}")
    
    for k in range(1, max_iter + 1):
        x_prev = x.copy()
        
        for i in range(n):
            # Suma usando valores ya actualizados (índices < i) y valores anteriores (índices > i)
            suma = (sum(A[i, j] * x[j] for j in range(i)) + 
                   sum(A[i, j] * x[j] for j in range(i + 1, n)))
            x[i] = (b[i] - suma) / A[i, i]
        
        # Verificar convergencia
        if np.linalg.norm(x - x_prev) < tol:
            logging.info(f"Convergencia alcanzada en {k} iteraciones")
            return x, trajectory
        
        trajectory.append(x.copy())
        logging.info(f"Iteración {k}: x = {x.flatten()}")
    
    logging.warning(f"No se alcanzó convergencia después de {max_iter} iteraciones")
    return x, trajectory


def main():
    """Función principal para probar los métodos."""
    
    # Definir el sistema de ecuaciones
    A = np.array([[1, 1], 
                  [-2, 1]], dtype=float)
    b = np.array([6, 0], dtype=float)
    x0 = np.array([5, 2], dtype=float)
    
    tol = 1e-6
    max_iter = 100
    
    print("=" * 50)
    print("SISTEMA DE ECUACIONES LINEALES")
    print("=" * 50)
    print(f"Matriz A:\n{A}")
    print(f"Vector b: {b}")
    print(f"Aproximación inicial x0: {x0}")
    print(f"Tolerancia: {tol}")
    print(f"Máximo de iteraciones: {max_iter}")
    print()
    
    # Resolver usando Gauss-Jacobi
    print("MÉTODO DE GAUSS-JACOBI:")
    print("-" * 30)
    try:
        x_jacobi, tray_jacobi = gauss_jacobi(A, b, x0, tol, max_iter)
        print(f"Solución: x = {x_jacobi[0, 0]:.6f}, y = {x_jacobi[1, 0]:.6f}")
        print(f"Número de iteraciones: {len(tray_jacobi)}")
    except Exception as e:
        print(f"Error en Gauss-Jacobi: {e}")
    
    print()
    
    # Resolver usando Gauss-Seidel
    print("MÉTODO DE GAUSS-SEIDEL:")
    print("-" * 30)
    try:
        x_seidel, tray_seidel = gauss_seidel(A, b, x0, tol, max_iter)
        print(f"Solución: x = {x_seidel[0, 0]:.6f}, y = {x_seidel[1, 0]:.6f}")
        print(f"Número de iteraciones: {len(tray_seidel)}")
    except Exception as e:
        print(f"Error en Gauss-Seidel: {e}")
    
    print()
    
    # Verificar solución con numpy
    print("VERIFICACIÓN CON NUMPY:")
    print("-" * 30)
    x_numpy = np.linalg.solve(A, b)
    print(f"Solución exacta: x = {x_numpy[0]:.6f}, y = {x_numpy[1]:.6f}")


if __name__ == "__main__":
    main()


def plot_iterations(tray_jacobi, tray_seidel, num_iterations=3):
    plt.figure(figsize=(10, 8))
    
    # Graficar las primeras num_iterations de cada método
    jacobi_x = [x[0,0] for x in tray_jacobi[:num_iterations+1]]
    jacobi_y = [x[1,0] for x in tray_jacobi[:num_iterations+1]]
    
    seidel_x = [x[0,0] for x in tray_seidel[:num_iterations+1]]
    seidel_y = [x[1,0] for x in tray_seidel[:num_iterations+1]]
    
    # Graficar las líneas del sistema de ecuaciones
    x = np.linspace(-1, 6, 100)
    plt.plot(x, 6-x, 'k--', label='x + y = 6')
    plt.plot(x, 2*x, 'k:', label='-2x + y = 0')
    
    # Graficar las trayectorias
    plt.plot(jacobi_x, jacobi_y, 'bo-', label='Gauss-Jacobi')
    plt.plot(seidel_x, seidel_y, 'ro-', label='Gauss-Seidel')
    
    # Marcar los puntos iniciales y finales
    plt.plot(jacobi_x[0], jacobi_y[0], 'bs', label='Punto inicial')
    plt.plot(jacobi_x[-1], jacobi_y[-1], 'b*', label='Última iteración Jacobi')
    plt.plot(seidel_x[-1], seidel_y[-1], 'r*', label='Última iteración Seidel')
    
    # Configurar el gráfico
    plt.grid(True)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Primeras {num_iterations} iteraciones de Gauss-Jacobi y Gauss-Seidel')
    plt.legend()
    plt.axis('equal')
    plt.show()

# Modificar la función main() agregando:
def main():
    # ... código existente ...
    
    try:
        x_jacobi, tray_jacobi = gauss_jacobi(A, b, x0, tol, max_iter)
        x_seidel, tray_seidel = gauss_seidel(A, b, x0, tol, max_iter)
        
        # Graficar las iteraciones
        plot_iterations(tray_jacobi, tray_seidel, 3)
        
    except Exception as e:
        print(f"Error: {e}")

[07-15 17:53:45][INFO] Iteración 0: x = [5. 2.]
[07-15 17:53:45][INFO] Iteración 1: x = [ 4. 10.]
[07-15 17:53:45][INFO] Iteración 2: x = [-4.  8.]
[07-15 17:53:45][INFO] Iteración 3: x = [-2. -8.]
[07-15 17:53:45][INFO] Iteración 4: x = [14. -4.]
[07-15 17:53:45][INFO] Iteración 5: x = [10. 28.]
[07-15 17:53:45][INFO] Iteración 6: x = [-22.  20.]
[07-15 17:53:45][INFO] Iteración 7: x = [-14. -44.]
[07-15 17:53:45][INFO] Iteración 8: x = [ 50. -28.]
[07-15 17:53:45][INFO] Iteración 9: x = [ 34. 100.]
[07-15 17:53:45][INFO] Iteración 10: x = [-94.  68.]
[07-15 17:53:45][INFO] Iteración 11: x = [ -62. -188.]
[07-15 17:53:45][INFO] Iteración 12: x = [ 194. -124.]
[07-15 17:53:45][INFO] Iteración 13: x = [130. 388.]
[07-15 17:53:45][INFO] Iteración 14: x = [-382.  260.]
[07-15 17:53:45][INFO] Iteración 15: x = [-254. -764.]
[07-15 17:53:45][INFO] Iteración 16: x = [ 770. -508.]
[07-15 17:53:45][INFO] Iteración 17: x = [ 514. 1540.]
[07-15 17:53:45][INFO] Iteración 18: x = [-1534.  1028.]
[

SISTEMA DE ECUACIONES LINEALES
Matriz A:
[[ 1.  1.]
 [-2.  1.]]
Vector b: [6. 0.]
Aproximación inicial x0: [5. 2.]
Tolerancia: 1e-06
Máximo de iteraciones: 100

MÉTODO DE GAUSS-JACOBI:
------------------------------
Solución: x = 3377699720527874.000000, y = -2251799813685244.000000
Número de iteraciones: 101

MÉTODO DE GAUSS-SEIDEL:
------------------------------


[07-15 17:53:45][INFO] Iteración 26: x = [-6.71088620e+07 -1.34217724e+08]
[07-15 17:53:45][INFO] Iteración 27: x = [1.3421773e+08 2.6843546e+08]
[07-15 17:53:45][INFO] Iteración 28: x = [-2.68435454e+08 -5.36870908e+08]
[07-15 17:53:45][INFO] Iteración 29: x = [5.36870914e+08 1.07374183e+09]
[07-15 17:53:45][INFO] Iteración 30: x = [-1.07374182e+09 -2.14748364e+09]
[07-15 17:53:45][INFO] Iteración 31: x = [2.14748365e+09 4.29496730e+09]
[07-15 17:53:45][INFO] Iteración 32: x = [-4.29496729e+09 -8.58993459e+09]
[07-15 17:53:45][INFO] Iteración 33: x = [8.58993459e+09 1.71798692e+10]
[07-15 17:53:45][INFO] Iteración 34: x = [-1.71798692e+10 -3.43597384e+10]
[07-15 17:53:45][INFO] Iteración 35: x = [3.43597384e+10 6.87194767e+10]
[07-15 17:53:45][INFO] Iteración 36: x = [-6.87194767e+10 -1.37438953e+11]
[07-15 17:53:45][INFO] Iteración 37: x = [1.37438953e+11 2.74877907e+11]
[07-15 17:53:45][INFO] Iteración 38: x = [-2.74877907e+11 -5.49755814e+11]
[07-15 17:53:45][INFO] Iteración 39: x 

Solución: x = -1267650600228229260759214850048.000000, y = -2535301200456458521518429700096.000000
Número de iteraciones: 101

VERIFICACIÓN CON NUMPY:
------------------------------
Solución exacta: x = 2.000000, y = 4.000000


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Métodos iterativos para resolver sistemas de ecuaciones lineales
Implementa los métodos de Gauss-Jacobi y Gauss-Seidel

Autor: z_tjona
Fecha: 19/07/2024
"""

import numpy as np
import logging
from typing import Tuple, List
from datetime import datetime
import matplotlib.pyplot as plt

# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    datefmt="%m-%d %H:%M:%S",
)


def validate_inputs(A: np.ndarray, b: np.ndarray, x0: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Valida y convierte los inputs a los tipos correctos.
    
    Args:
        A: Matriz de coeficientes
        b: Vector de términos independientes
        x0: Vector de aproximación inicial
        
    Returns:
        Tupla con A, b, x0 validados y convertidos
        
    Raises:
        ValueError: Si las dimensiones no son correctas
    """
    # Convertir A a numpy array
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)
    
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A debe ser cuadrada (n×n)")
    
    # Convertir b a numpy array
    if not isinstance(b, np.ndarray):
        b = np.array(b, dtype=float)
    
    if b.ndim == 1:
        b = b.reshape(-1, 1)
    
    if b.shape[0] != A.shape[0]:
        raise ValueError("El vector b debe tener n elementos")
    
    # Convertir x0 a numpy array
    if not isinstance(x0, np.ndarray):
        x0 = np.array(x0, dtype=float)
    
    if x0.ndim == 1:
        x0 = x0.reshape(-1, 1)
    
    if x0.shape[0] != A.shape[0]:
        raise ValueError("El vector x0 debe tener n elementos")
    
    return A, b, x0


def gauss_jacobi(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                tol: float = 1e-6, max_iter: int = 100) -> Tuple[np.ndarray, List[np.ndarray]]:
    """
    Resuelve el sistema Ax = b usando el método de Gauss-Jacobi.
    
    En cada iteración, usa los valores de x de la iteración anterior
    para calcular todos los componentes del nuevo vector.
    
    Args:
        A: Matriz de coeficientes (n×n)
        b: Vector de términos independientes (n×1)
        x0: Vector de aproximación inicial (n×1)
        tol: Tolerancia para la convergencia
        max_iter: Número máximo de iteraciones
        
    Returns:
        Tupla con:
        - Vector solución
        - Lista con la trayectoria de iteraciones
    """
    A, b, x0 = validate_inputs(A, b, x0)
    
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    
    logging.info(f"Iteración 0: x = {x.flatten()}")
    
    for k in range(1, max_iter + 1):
        x_new = np.zeros((n, 1))
        
        for i in range(n):
            # Suma de todos los elementos excepto el diagonal
            suma = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - suma) / A[i, i]
        
        # Verificar convergencia
        if np.linalg.norm(x_new - x) < tol:
            logging.info(f"Convergencia alcanzada en {k} iteraciones")
            return x_new, trajectory
        
        x = x_new.copy()
        trajectory.append(x.copy())
        logging.info(f"Iteración {k}: x = {x.flatten()}")
    
    logging.warning(f"No se alcanzó convergencia después de {max_iter} iteraciones")
    return x, trajectory


def gauss_seidel(A: np.ndarray, b: np.ndarray, x0: np.ndarray, 
                tol: float = 1e-6, max_iter: int = 100) -> Tuple[np.ndarray, List[np.ndarray]]:
    """
    Resuelve el sistema Ax = b usando el método de Gauss-Seidel.
    
    En cada iteración, usa los valores más recientes de x conforme
    se van calculando (convergencia más rápida que Jacobi).
    
    Args:
        A: Matriz de coeficientes (n×n)
        b: Vector de términos independientes (n×1)
        x0: Vector de aproximación inicial (n×1)
        tol: Tolerancia para la convergencia
        max_iter: Número máximo de iteraciones
        
    Returns:
        Tupla con:
        - Vector solución
        - Lista con la trayectoria de iteraciones
    """
    A, b, x0 = validate_inputs(A, b, x0)
    
    n = A.shape[0]
    x = x0.copy()
    trajectory = [x.copy()]
    
    logging.info(f"Iteración 0: x = {x.flatten()}")
    
    for k in range(1, max_iter + 1):
        x_prev = x.copy()
        
        for i in range(n):
            # Suma usando valores ya actualizados (índices < i) y valores anteriores (índices > i)
            suma = (sum(A[i, j] * x[j] for j in range(i)) + 
                   sum(A[i, j] * x[j] for j in range(i + 1, n)))
            x[i] = (b[i] - suma) / A[i, i]
        
        # Verificar convergencia
        if np.linalg.norm(x - x_prev) < tol:
            logging.info(f"Convergencia alcanzada en {k} iteraciones")
            return x, trajectory
        
        trajectory.append(x.copy())
        logging.info(f"Iteración {k}: x = {x.flatten()}")
    
    logging.warning(f"No se alcanzó convergencia después de {max_iter} iteraciones")
    return x, trajectory


def plot_convergence(tray_jacobi: List[np.ndarray], tray_seidel: List[np.ndarray], 
                    A: np.ndarray, b: np.ndarray, num_iterations: int = 3):
    """
    Genera dos gráficos mostrando la convergencia de ambos métodos.
    
    Args:
        tray_jacobi: Trayectoria de iteraciones del método Jacobi
        tray_seidel: Trayectoria de iteraciones del método Gauss-Seidel
        A: Matriz de coeficientes
        b: Vector de términos independientes
        num_iterations: Número de iteraciones a mostrar
    """
    # Crear figura con dos subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Calcular la solución exacta
    x_exact = np.linalg.solve(A, b)
    
    # Generar puntos para las líneas del sistema
    x_range = np.linspace(-1, 7, 100)
    
    # Para el sistema Ax = b donde A = [[1, 1], [-2, 1]] y b = [6, 0]
    # Ecuación 1: x + y = 6  -> y = 6 - x
    # Ecuación 2: -2x + y = 0  -> y = 2x
    y1 = 6 - x_range  # Primera ecuación
    y2 = 2 * x_range  # Segunda ecuación
    
    # Configurar gráfico para Gauss-Jacobi
    ax1.plot(x_range, y1, 'k--', alpha=0.7, label='x + y = 6')
    ax1.plot(x_range, y2, 'k:', alpha=0.7, label='-2x + y = 0')
    ax1.plot(x_exact[0], x_exact[1], 'g*', markersize=15, label='Solución exacta')
    
    # Plotear trayectoria Jacobi
    jacobi_points = tray_jacobi[:num_iterations+1]
    jacobi_x = [point[0, 0] for point in jacobi_points]
    jacobi_y = [point[1, 0] for point in jacobi_points]
    
    ax1.plot(jacobi_x, jacobi_y, 'bo-', linewidth=2, markersize=8, label='Trayectoria Jacobi')
    
    # Numerar los puntos
    for i, (x, y) in enumerate(zip(jacobi_x, jacobi_y)):
        ax1.annotate(f'{i}', (x, y), xytext=(5, 5), textcoords='offset points',
                    fontsize=10, bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue'))
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title('Método de Gauss-Jacobi')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_xlim(-1, 7)
    ax1.set_ylim(-1, 7)
    ax1.set_aspect('equal')
    
    # Configurar gráfico para Gauss-Seidel
    ax2.plot(x_range, y1, 'k--', alpha=0.7, label='x + y = 6')
    ax2.plot(x_range, y2, 'k:', alpha=0.7, label='-2x + y = 0')
    ax2.plot(x_exact[0], x_exact[1], 'g*', markersize=15, label='Solución exacta')
    
    # Plotear trayectoria Seidel
    seidel_points = tray_seidel[:num_iterations+1]
    seidel_x = [point[0, 0] for point in seidel_points]
    seidel_y = [point[1, 0] for point in seidel_points]
    
    ax2.plot(seidel_x, seidel_y, 'ro-', linewidth=2, markersize=8, label='Trayectoria Seidel')
    
    # Numerar los puntos
    for i, (x, y) in enumerate(zip(seidel_x, seidel_y)):
        ax2.annotate(f'{i}', (x, y), xytext=(5, 5), textcoords='offset points',
                    fontsize=10, bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral'))
    
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_title('Método de Gauss-Seidel')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    ax2.set_xlim(-1, 7)
    ax2.set_ylim(-1, 7)
    ax2.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar información de convergencia
    print(f"\nINFORMACIÓN DE CONVERGENCIA:")
    print(f"Solución exacta: x = {x_exact[0]:.6f}, y = {x_exact[1]:.6f}")
    print(f"\nPrimeras {num_iterations} iteraciones:")
    print(f"{'Iter':<4} {'Jacobi (x, y)':<20} {'Seidel (x, y)':<20}")
    print("-" * 50)
    
    max_iter = min(num_iterations + 1, len(tray_jacobi), len(tray_seidel))
    for i in range(max_iter):
        jx, jy = tray_jacobi[i][0, 0], tray_jacobi[i][1, 0]
        sx, sy = tray_seidel[i][0, 0], tray_seidel[i][1, 0]
        print(f"{i:<4} ({jx:.4f}, {jy:.4f})       ({sx:.4f}, {sy:.4f})")


def plot_error_convergence(tray_jacobi: List[np.ndarray], tray_seidel: List[np.ndarray], 
                          A: np.ndarray, b: np.ndarray):
    """
    Grafica la evolución del error en cada iteración.
    """
    x_exact = np.linalg.solve(A, b)
    
    # Calcular errores
    errors_jacobi = [np.linalg.norm(x.flatten() - x_exact) for x in tray_jacobi]
    errors_seidel = [np.linalg.norm(x.flatten() - x_exact) for x in tray_seidel]
    
    plt.figure(figsize=(10, 6))
    plt.semilogy(range(len(errors_jacobi)), errors_jacobi, 'bo-', label='Gauss-Jacobi')
    plt.semilogy(range(len(errors_seidel)), errors_seidel, 'ro-', label='Gauss-Seidel')
    
    plt.xlabel('Iteración')
    plt.ylabel('Error (escala logarítmica)')
    plt.title('Convergencia del Error')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()


def main():
    """Función principal para probar los métodos."""
    
    # Definir el sistema de ecuaciones
    A = np.array([[1, 1], 
                  [-2, 1]], dtype=float)
    b = np.array([6, 0], dtype=float)
    x0 = np.array([5, 2], dtype=float)
    
    tol = 1e-6
    max_iter = 100
    
    print("=" * 50)
    print("SISTEMA DE ECUACIONES LINEALES")
    print("=" * 50)
    print(f"Matriz A:\n{A}")
    print(f"Vector b: {b}")
    print(f"Aproximación inicial x0: {x0}")
    print(f"Tolerancia: {tol}")
    print(f"Máximo de iteraciones: {max_iter}")
    print()
    
    # Resolver usando ambos métodos
    print("MÉTODO DE GAUSS-JACOBI:")
    print("-" * 30)
    try:
        x_jacobi, tray_jacobi = gauss_jacobi(A, b, x0, tol, max_iter)
        print(f"Solución: x = {x_jacobi[0, 0]:.6f}, y = {x_jacobi[1, 0]:.6f}")
        print(f"Número de iteraciones: {len(tray_jacobi)}")
    except Exception as e:
        print(f"Error en Gauss-Jacobi: {e}")
        return
    
    print()
    
    print("MÉTODO DE GAUSS-SEIDEL:")
    print("-" * 30)
    try:
        x_seidel, tray_seidel = gauss_seidel(A, b, x0, tol, max_iter)
        print(f"Solución: x = {x_seidel[0, 0]:.6f}, y = {x_seidel[1, 0]:.6f}")
        print(f"Número de iteraciones: {len(tray_seidel)}")
    except Exception as e:
        print(f"Error en Gauss-Seidel: {e}")
        return
    
    print()
    
    # Verificar solución con numpy
    print("VERIFICACIÓN CON NUMPY:")
    print("-" * 30)
    x_numpy = np.linalg.solve(A, b)
    print(f"Solución exacta: x = {x_numpy[0]:.6f}, y = {x_numpy[1]:.6f}")
    
    # Generar gráficos
    print("\nGenerando gráficos...")
    plot_convergence(tray_jacobi, tray_seidel, A, b, num_iterations=3)
    plot_error_convergence(tray_jacobi, tray_seidel, A, b)


if __name__ == "__main__":
    main()